# VacationPy

In [17]:
import hvplot.pandas
from hvplot import save
import pandas as pd
import requests
from api_keys import GEO_API_KEY

In [2]:
city_df = pd.read_csv("output_data/cities.csv", index_col='City_ID')
city_df['Max Temp'] = round((9/5) * (city_df['Max Temp'] - 273.15) + 32, 2) 
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,kenai,60.5544,-151.2583,42.89,87,100,3.09,US,1714541374
1,margaret river,-33.9500,115.0667,72.09,57,100,6.99,AU,1714541106
2,kingston,17.9970,-76.7936,83.35,61,75,4.12,JM,1714541313
3,albany,42.6001,-73.9662,58.26,82,100,1.77,US,1714541254
4,sanming,26.2486,117.6186,65.26,91,100,1.20,CN,1714541377


In [3]:
%%capture --no-display

city_plot = city_df.hvplot.points('Lng', 'Lat', geo=True, color='City', size='Humidity', alpha=0.5, scale=1.25, frame_width = 800, frame_height = 500, tiles='OSM')
                    
city_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

## Find ideal weather conditions

In [4]:
#First lets consider what my ideal weather conditions are:
    # Not too humid, 0-50 Avearge temperature 40-80  cloudiness between 0-50
    
city_df.sample(25)
hotel_df = city_df.query("Humidity >= 0 & Humidity <=50 & `Max Temp` >= 60 & `Max Temp`<=80 & Cloudiness >= 5 & Cloudiness <= 50")[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()
hotel_df['Hotel Name'] = " "
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
77,yantai,CN,37.5333,121.4000,43,
85,tolanaro,MG,-25.0319,46.9987,43,
107,prieska,ZA,-29.6641,22.7474,36,
222,al jawf,SA,29.5000,38.7500,48,
267,mary,TM,37.0000,62.5000,14,
306,al qubbah,LY,32.7616,22.2424,44,
328,siwa oasis,EG,29.2041,25.5195,37,
383,taoudenni,ML,22.6783,-3.9836,12,
405,queenstown,ZA,-31.8976,26.8753,46,


In [5]:
# Find hotel 10,000 metres of the coordinates.

radius = 10000
params = {
    'categories': 'accommodation.hotel',
    'limit': 1,
    'apiKey': GEO_API_KEY
}

# Print a message to follow up the hotel search
print("Starting hotel search")

for idx, row in hotel_df.iterrows():
    lt = row[2]
    ln = row[3]
    
    params['filter'] = f'circle:{ln},{lt},{radius}'
    params['bias'] = f'proximity:{ln},{lt}'
    
    url = "https://api.geoapify.com/v2/places"
    name_address = requests.get(url, params).json()
    
    try:
        hotel_df.loc[idx, 'Hotel Name'] = name_address['features'][0]['properties']['name']
    except:
        hotel_df.loc[idx, 'Hotel Name'] = 'No hotel found'
    
    print(f"{hotel_df.loc[idx, 'City']} - nearest hotel: {hotel_df.loc[idx, 'Hotel Name']}")
    
hotel_df
    

Starting hotel search
yantai - nearest hotel: Hilton Yantai
tolanaro - nearest hotel: Hôtel Mahavokey
prieska - nearest hotel: No hotel found
al jawf - nearest hotel: No hotel found
mary - nearest hotel: No hotel found
al qubbah - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
taoudenni - nearest hotel: No hotel found
queenstown - nearest hotel: Queens Hotel
aoulef - nearest hotel: دار الضياف
kayenta - nearest hotel: Wetherill Inn
dalnegorsk - nearest hotel: Дальнегорск
ad dawadimi - nearest hotel: No hotel found
tengrela - nearest hotel: No hotel found
bardai - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
City_ID,,,,,,
77,yantai,CN,37.5333,121.4000,43,Hilton Yantai
85,tolanaro,MG,-25.0319,46.9987,43,Hôtel Mahavokey
107,prieska,ZA,-29.6641,22.7474,36,No hotel found
222,al jawf,SA,29.5000,38.7500,48,No hotel found
267,mary,TM,37.0000,62.5000,14,No hotel found
306,al qubbah,LY,32.7616,22.2424,44,No hotel found
328,siwa oasis,EG,29.2041,25.5195,37,فندق الكيلانى
383,taoudenni,ML,22.6783,-3.9836,12,No hotel found
405,queenstown,ZA,-31.8976,26.8753,46,Queens Hotel


## HVPLOT with hotels only on cities within ideal weather conditions for a vacation.

In [19]:
%%capture --no-display

hotel_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, color='City', size='Humidity',
                                    alpha=0.5, scale=2.5, frame_width = 800, frame_height = 500,
                                    tiles='OSM', hover_cols= ['Hotel Name', 'Country'])

hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)